In [2]:
%load_ext autoreload
%autoreload 2
%run ./initializer

Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [3]:
import pandas as pd
from lgbm_ranker import LGBMRankerModel, LGBMRanker
from feature_tester import FeatureTester
import numpy as np

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_df = pd.read_parquet("data/training_set_processed.parquet")

In [5]:
lgbm_class_model = LGBMRankerModel(
    df = train_df,              # Only initialized to get column names
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[5],
    n_estimators=1000,
    learning_rate=0.05,
)

In [ ]:
feature_tester = FeatureTester(lgbm_class_model)
list_of_categorical_features_threshold = [{"srch_destination_id": 1000}, {"srch_destination_id": 500}, {"srch_destination_id": 2500}, {"srch_destination_id": 5000}, 
                                          {"site_id": 5000, "visitor_location_country_id": 1000, "prop_country_id": 1000, "srch_destination_id": 5000}, 
                                          {"site_id": 2500, "visitor_location_country_id": 500, "prop_country_id": 500, "srch_destination_id": 5000}, {"site_id": 2500, "visitor_location_country_id": 500, "prop_country_id": 500, "srch_destination_id": 2500}]
feature_tester.test_cardinal_reduction_methods(list_of_categorical_features_threshold)


In [ ]:
categorical_features_threshold = {"site_id": 5000, "visitor_location_country_id": 1000, "prop_country_id": 1000, "srch_destination_id": 5000}
feature_tester = FeatureTester(lgbm_class_model, categorical_features_threshold= categorical_features_threshold)

impute_null_columns_methods = [{"visitor_hist_starrating": "median", "visitor_hist_adr_usd": "mean", "prop_review_score": "median", "orig_destination_distance": "mean", "srch_query_affinity_score":"mean", "prop_log_historical_price":"mean", "prop_location_score2": "mean"},
                               {'prop_review_score': 'median', 'orig_destination_distance': 'mean'}]
feature_tester.test_na_fill_method(impute_null_columns_methods)

In [ ]:
train_df_light_feature_filtered = FeatureTester(lgbm_class_model).light_feature_filtering(train_df)

In [ ]:
feature_tester = FeatureTester(lgbm_class_model)
feature_tester.run_light_hpo(train_df, n_trials=30)

[I 2025-05-12 15:55:26,441] A new study created in memory with name: no-name-f41311b2-3f3e-489a-81ea-a03e45799476
[I 2025-05-12 15:56:35,982] Trial 0 finished with value: 0.4140832050579985 and parameters: {'n_estimators': 102, 'learning_rate': 0.14535583580196887, 'num_leaves': 86, 'min_child_samples': 38, 'reg_alpha': 0.06734493547372118, 'reg_lambda': 0.00048091355230024755}. Best is trial 0 with value: 0.4140832050579985.
[I 2025-05-12 15:58:20,877] Trial 1 finished with value: 0.20457432831998423 and parameters: {'n_estimators': 226, 'learning_rate': 0.12082418412676686, 'num_leaves': 47, 'min_child_samples': 19, 'reg_alpha': 0.039247288416794264, 'reg_lambda': 0.7155363330024246}. Best is trial 0 with value: 0.4140832050579985.
[W 2025-05-12 15:59:59,020] Trial 2 failed with parameters: {'n_estimators': 249, 'learning_rate': 0.014866468908437106, 'num_leaves': 15, 'min_child_samples': 32, 'reg_alpha': 0.9910659790005502, 'reg_lambda': 0.12952343754977472} because of the following

KeyboardInterrupt: 